# Dialogue
>  Everything you've built so far has statelessly mapped intents to actions and responses. It's amazing how far you can get with that! But to build more sophisticated bots you will always want to add some statefulness. That's what you'll do here, as you build a chatbot that helps users order coffee.

- toc: true 
- badges: true
- comments: true
- author: Lucas Nunes
- categories: [Datacamp]
- image: images/datacamp/___

> Note: This is a summary of the course's chapter 4 exercises "Building Chatbots in Python" at datacamp. <br>[Github repo](https://github.com/lnunesAI/Datacamp/) / [Course link](https://www.datacamp.com/tracks/machine-learning-scientist-with-python)

## Why statefulness is key

### Form filling

<div class=""><p>You'll often want your bot to guide users through a series of steps, such as when they're placing an order.</p>
<p>In this exercise, you'll begin building a bot that lets users order coffee. They can choose between two types: Colombian and Kenyan. If the user provides unexpected input, your bot will handle this differently depending on where they are in the flow.</p>
<p>Your job here is to identify the appropriate state and next state based on the intents and response messages provided. For example, if the intent is <code>"order"</code>, then the state changes from <code>INIT</code> to <code>CHOOSE_COFFEE</code>.</p>
<p>A function <code>send_message(policy, state, message)</code> has already been defined for you.
It takes the policy, the current state, and message as arguments, and returns the new state as a result. Additionally, an <code>interpret(message)</code> function, similar to the one Alan described in the video, has been pre-defined for you.</p></div>

In [ ]:
def send_message(policy, state, message):
    print("USER : {}".format(message))
    new_state, response = respond(policy, state, message)
    print("BOT : {}".format(response))
    return new_state

In [ ]:
def respond(policy, state, message):
    (new_state, response) = policy[(state, interpret(message))]
    return new_state, response

Instructions
<ul>
<li>Define three states: <code>INIT</code> with value <code>0</code>, <code>CHOOSE_COFFEE</code> with value <code>1</code>, and <code>ORDERED</code> with value <code>2</code>.</li>
<li>Create a dictionary called <code>policy</code> with tuples as keys and values. Each key is a tuple containing a state and an intent, and each value is a tuple containing the next state and the response message. The messages have been filled in for you. Your job is to fill in the states.</li>
<li>Instantiate a variable <code>state</code> with the value <code>INIT</code>.</li>
<li>For each of the messages, call the <code>send_message()</code> function, passing in the <code>policy</code>, <code>state</code>, and <code>message</code>.</li>
</ul>

In [ ]:
# Define the INIT state
INIT = 0

# Define the CHOOSE_COFFEE state
CHOOSE_COFFEE = 1

# Define the ORDERED state
ORDERED = 2

# Define the policy rules
policy = {
    (INIT, "order"): (CHOOSE_COFFEE, "ok, Colombian or Kenyan?"),
    (INIT, "none"): (INIT, "I'm sorry - I'm not sure how to help you"),
    (CHOOSE_COFFEE, "specify_coffee"): (ORDERED, "perfect, the beans are on their way!"),
    (CHOOSE_COFFEE, "none"): (CHOOSE_COFFEE, "I'm sorry - would you like Colombian or Kenyan?"),
}

# Create the list of messages
messages = [
    "I'd like to become a professional dancer",
    "well then I'd like to order some coffee",
    "my favourite animal is a zebra",
    "kenyan"
]

# Call send_message() for each message
state = INIT
for message in messages:    
    state = send_message(policy, state, message)

### Asking contextual questions

<div class=""><p>Sometimes your users need some help! They will have questions and expect the bot to help them.</p>
<p>In this exercise, you'll allow users to ask the coffee bot to explain the steps to them.
As in the previous exercise, the answer they get will depend on where they are in the flow.</p></div>

Instructions
<ul>
<li>Add two rules to your <code>policy_rules</code> to handle the intent <code>"ask_explanation"</code> when in the states <code>INIT</code> or <code>CHOOSE_COFFEE</code>.</li>
<li>Inside the <code>send_messages()</code> function, call the <code>send_message()</code> function with <code>state</code> and <code>msg</code> as arguments to define the new <code>state</code>. Then, hit 'Submit Answer' to send the messages and see the bot's responses.</li>
</ul>

In [ ]:
# Define the states
INIT=0 
CHOOSE_COFFEE=1
ORDERED=2

# Define the policy rules dictionary
policy_rules = {
    (INIT, "ask_explanation"): (INIT, "I'm a bot to help you order coffee beans"),
    (INIT, "order"): (CHOOSE_COFFEE, "ok, Colombian or Kenyan?"),
    (CHOOSE_COFFEE, "specify_coffee"): (ORDERED, "perfect, the beans are on their way!"),
    (CHOOSE_COFFEE, "ask_explanation"): (CHOOSE_COFFEE, "We have two kinds of coffee beans - the Kenyan ones make a slightly sweeter coffee, and cost $6. The Brazilian beans make a nutty coffee and cost $5.")    
}

# Define send_messages()
def send_messages(messages):
    state = INIT
    for msg in messages:
        state = send_message(state, msg)

# Send the messages
send_messages([
    "what can you do for me?",
    "well then I'd like to order some coffee",
    "what do you mean by that?",
    "kenyan"
])

**Your bot can now modify its answers by considering context**

### Dealing with rejection

<div class=""><p>What happens if you make a suggestion to your user and they don't like it? Your bot will look really silly if it makes the same suggestion again right away.</p>
<p>Here, you're going to modify your <code>respond()</code> function so that it accepts and returns 4 arguments: </p>
<ul>
<li>The user message as an argument, and the bot response as the first return value.</li>
<li>A dictionary <code>params</code> including the entities the user has specified.</li>
<li>A <code>prev_suggestions</code> list. When passed to <code>respond()</code>, this should contain the suggestions made in the previous bot message. When returned by <code>respond()</code>, it should contain the current suggestions.</li>
<li>An <code>excluded</code> list, which contains all of the results your user has already explicitly rejected.</li>
</ul>
<p>Your function should add the previous suggestions to the excluded list whenever it receives a <code>"deny"</code> intent. It should also filter out excluded suggestions from the response.</p></div>

In [ ]:
def find_hotels(params, excluded):
    query = 'SELECT * FROM hotels'
    if len(params) > 0:
        filters = ["{}=?".format(k) for k in params] +                  ["name!='?'".format(k) for k in excluded] 
        query += " WHERE " + " and ".join(filters)
    t = tuple(params.values())
    
    # open connection to DB
    conn = sqlite3.connect('hotels.db')
    # create a cursor
    c = conn.cursor()
    c.execute(query, t)
    return c.fetchall()

Instructions
<ul>
<li>Define a <code>respond()</code> function with 4 arguments: <code>message</code>, <code>params</code>, <code>prev_suggestions</code>, and <code>excluded</code>.</li>
<li>Interpret the <code>message</code> and store the result in <code>parse_data</code>.</li>
<li>The value of the <code>"intent"</code> key of <code>parse_data</code> is itself a dictionary of key-value pairs. Assign <code>parse_data["intent"]["name"]</code> to <code>intent</code>, and <code>parse_data["entities"]</code> to <code>entities</code>.</li>
<li>If the <code>intent</code> is <code>"deny"</code>, use the <code>.extend()</code> method of the <code>excluded</code> list to add <code>prev_suggestions</code> to it.</li>
<li>Initialize the empty <code>params</code> dictionary and empty <code>suggestions</code> and <code>excluded</code> lists. Then, hit 'Submit Answer' to send the messages to the bot.</li>
</ul>

In [ ]:
# Define respond()
def respond(message, params, prev_suggestions, excluded):
    # Interpret the message
    parse_data = interpret(message)
    # Extract the intent
    intent = parse_data["intent"]["name"]
    # Extract the entities
    entities = parse_data["entities"]
    # Add the suggestion to the excluded list if intent is "deny"
    if intent == "deny":
        excluded.extend(prev_suggestions)
    # Fill the dictionary with entities
    for ent in entities:
        params[ent["entity"]] = str(ent["value"])
    # Find matching hotels
    results = [
        r 
        for r in find_hotels(params, excluded) 
        if r[0] not in excluded
    ]
    # Extract the suggestions
    names = [r[0] for r in results]
    n = min(len(results), 3)
    suggestions = names[:2]
    return responses[n].format(*names), params, suggestions, excluded

# Initialize the empty dictionary and lists
params, suggestions, excluded = {}, [], []

# Send the messages
for message in ["I want a mid range hotel", "no that doesn't work for me"]:
    print("USER: {}".format(message))
    response, params, suggestions, excluded = respond(message, params, suggestions, excluded)
    print("BOT: {}".format(response))

**Your bot can now handle negative feedback gracefully.**

## Asking questions & queuing answers

### Pending actions I

<div class=""><p>You can really improve the user experience of your bot by asking the user simple yes or no follow-up questions. One easy way to handle these follow-ups is to define <em>pending</em> actions which get executed as soon as the user says "yes", and wiped if the user says "no". </p>
<p>In this exercise, you're going to define a <code>policy()</code> function which takes the <code>intent</code> as its sole argument and returns two values: The next action to take and a pending action. The policy function should return this pending action when a "yes" or <code>"affirm"</code> intent is returned and should wipe the pending actions if a "no" or <code>"deny"</code> intent is returned.</p>
<p>Here, the <code>interpret(message)</code> function has been defined for you such that if <code>"yes"</code> is in the message, <code>"affirm"</code> is returned, and if <code>"no"</code> is in the message, then <code>"deny"</code> is returned.</p></div>

Instructions
<ul>
<li>Define a function called <code>policy()</code> which takes <code>intent</code> as its argument.</li>
<li>If the <code>intent</code> is <code>"affirm"</code>, return a <code>"do_pending"</code> action and <code>None</code>.</li>
<li>If the <code>intent</code> is <code>"deny"</code>, return a <code>"Ok"</code> action and <code>None</code>.</li>
</ul>

In [ ]:
# Define policy()
def policy(intent):
    # Return "do_pending" if the intent is "affirm"
    if intent == "affirm":
        return "do_pending", None
    # Return "Ok" if the intent is "deny"
    if intent == "deny":
        return "Ok", None
    if intent == "order":
        return "Unfortunately, the Kenyan coffee is currently out of stock, would you like to order the Brazilian beans?", "Alright, I've ordered that for you!"

**With a policy() function defined, you can now incorporate it into a send_message() function.**

### Pending actions II

<div class=""><p>Having defined your <code>policy()</code> function, it's now time to write a <code>send_message()</code> function which takes both a <code>pending</code> action and a <code>message</code> as its arguments and leverages the <code>policy()</code> function to determine the bot's response.</p>
<p>Your <code>policy(intent)</code> function from the previous exercise has been pre-loaded.</p></div>

In [ ]:
def interpret(message):
    msg = message.lower()
    if 'order' in msg:
        return 'order'
    elif 'yes' in msg:
        return 'affirm'
    elif 'no' in msg:
        return 'deny'
    return 'none'

Instructions
<ul>
<li>Define a function called <code>send_message()</code> which takes in two arguments: <code>pending</code> and <code>message</code>.</li>
<li>Pass in the interpretation of <code>message</code> as an argument to <code>policy()</code> and unpack the result into the variables <code>action</code> and <code>pending_action</code>.</li>
<li>If the <code>action</code> is <code>"do_pending"</code> and <code>pending</code> is not <code>None</code>, print the <code>pending</code> response. Else, print the <code>action</code>. </li>
<li>Inside the definition of the <code>send_messages()</code> function, call your <code>send_message()</code> function with <code>pending</code> and <code>msg</code> as arguments. Then, hit 'Submit Answer' to send the messages and see the results.</li>
</ul>

In [ ]:
# Define send_message()
def send_message(pending, message):
    print("USER : {}".format(message))
    action, pending_action = policy(interpret(message))
    if action == "do_pending" and pending is not None:
        print("BOT : {}".format(pending))
    else:
        print("BOT : {}".format(action))
    return pending_action
    
# Define send_messages()
def send_messages(messages):
    pending = None
    for msg in messages:
        pending = send_message(pending, msg)

# Send the messages
send_messages([
    "I'd like to order some coffee",
    "ok yes please"
])

**Your bot can now follow up on its own suggestions!**

### Pending state transitions

<div class=""><p>You'll often need to briefly deviate from the flow of a conversation, for example to authenticate a user, before returning to the topic of discussion. </p>
<p>In these cases, it's often simpler - and easier to debug - if you save some actions/states as <em>pending</em> rather than adding ever more complicated rules.</p>
<p>Here, you're going to define a <code>policy_rules</code> dictionary, where the keys are tuples of the current state and the received intent, and the values are tuples of the next state, the bot's response, and a state for which to set a pending transition.</p></div>

In [ ]:
def interpret(message):
    msg = message.lower()
    if 'order' in msg:
        return 'order'
    if 'kenyan' in msg or 'colombian' in msg:
        return 'specify_coffee'
    if any([d in msg for d in string.digits]):
        return 'number'    
    return 'none'

In [ ]:
def send_message(state, pending, message):
    print("USER : {}".format(message))
    new_state, response, pending_state = policy_rules[(state, interpret(message))]
    print("BOT : {}".format(response))
    if pending is not None:
        new_state, response, pending_state = policy_rules[pending]
        print("BOT : {}".format(response))        
    if pending_state is not None:
        pending = (pending_state, interpret(message))
    return new_state, pending

Instructions
<ul>
<li>Complete the <code>policy_rules</code> dictionary by filling in the values: <ul>
<li>A user starts in the <code>INIT</code> state. </li>
<li>If the user is in the <code>INIT</code> state and tries to place an order, you should ask for their number and create a pending transition to the <code>AUTHED</code> state. </li>
<li>This is the only policy rule which creates a pending transition, so the others simply have a pending state value of <code>None</code>.</li></ul></li>
<li>The <code>pending</code> state has been added as the second argument of the <code>send_message()</code> function, which now returns the new state as well as the pending state. Call this <code>send_message()</code> function inside <code>send_messages()</code>, unpacking the output into the variables <code>state</code> and <code>pending</code>. </li>
<li>Hit 'Submit Answer' to send the messages to the bot!</li>
</ul>

In [ ]:
# Define the states
INIT=0
AUTHED=1
CHOOSE_COFFEE=2
ORDERED=3

# Define the policy rules
policy_rules = {
    (INIT, "order"): (INIT, "you'll have to log in first, what's your phone number?", AUTHED),
    (INIT, "number"): (AUTHED, "perfect, welcome back!", None),
    (AUTHED, "order"): (CHOOSE_COFFEE, "would you like Colombian or Kenyan?", None),    
    (CHOOSE_COFFEE, "specify_coffee"): (ORDERED, "perfect, the beans are on their way!", None)
}

# Define send_messages()
def send_messages(messages):
    state = INIT
    pending = None
    for msg in messages:
        state, pending = send_message(state, pending, msg)

# Send the messages
send_messages([
    "I'd like to order some coffee",
    "555-1234",
    "kenyan"
])

### Putting it all together I

<div class=""><p>It's time to put everything you've learned in the course together by combining the coffee ordering bot with the ELIZA rules from chapter 1.</p>
<p>To begin, you'll define a function called <code>chitchat_response()</code>, which calls the predefined function <code>match_rule()</code> from back in chapter 1. This returns a response if the message matched an ELIZA template, and otherwise, <code>None</code>.</p>
<p>The ELIZA rules are contained in a dictionary called <code>eliza_rules</code>.</p></div>

In [ ]:
def match_rule(rules, message):
    for pattern, responses in rules.items():
        match = re.search(pattern, message)
        if match is not None:
            response = random.choice(responses)
            var = match.group(1) if '{0}' in response else None
            return response, var
    return "default", None

In [ ]:
def replace_pronouns(message):
    message = message.lower()
    if 'me' in message:
        return re.sub('me', 'you', message)
    if 'i' in message:
        return re.sub('i', 'you', message)
    elif 'my' in message:
        return re.sub('my', 'your', message)
    elif 'your' in message:
        return re.sub('your', 'my', message)
    elif 'you' in message:
        return re.sub('you', 'me', message)
    return message

Instructions
<ul>
<li>Define a <code>chitchat_response()</code> function which takes in a <code>message</code> argument.</li>
<li>Call the <code>match_rule()</code> function with <code>eliza_rules</code> and <code>message</code> as arguments. Unpack the output into <code>response</code> and <code>phrase</code>.</li>
<li>If the response is <code>"default"</code>, return <code>None</code>.</li>
<li>If <code>"{0}"</code> is in the response, replace the pronouns of the <code>phrase</code> using <code>replace_pronouns()</code>, and then include the <code>phrase</code> in the <code>response</code> by using <code>.format()</code> on <code>response</code>.</li>
</ul>

In [ ]:
# Define chitchat_response()
def chitchat_response(message):
    # Call match_rule()
    response, phrase = match_rule(eliza_rules, message)
    # Return none if response is "default"
    if response == "default":
        return None
    if '{0}' in response:
        # Replace the pronouns of phrase
        phrase = replace_pronouns(phrase)
        # Calculate the response
        response = response.format(phrase)
    return response

**You've put it all together and your bot can now interleave chit-chat and functional conversation.**

### Putting it all together II

<p>With your <code>chitchat_response(message)</code> function defined, the next step is to define a <code>send_message()</code> function. This function should first call <code>chitchat_response(message)</code> and only use the coffee bot policy if there is no matching message.</p>

In [ ]:
policy_rules = {(0, 'number'): (1, 'perfect, welcome back!', None),
 (0, 'order'): (0,
  "you'll have to log in first, what's your phone number?",
  1),
 (1, 'order'): (2, 'would you like Colombian or Kenyan?', None),
 (2, 'specify_coffee'): (3, 'perfect, the beans are on their way!', None)}

In [ ]:
def chitchat_response(message):
    response, var = match_rule(eliza_rules, message)
    if response == 'default':
        return None
    if '{0}' in response:
        var = replace_pronouns(var)
        response = response.format(var)
    return response

Instructions
<ul>
<li>Define a <code>send_message()</code> function which takes in 3 arguments: <code>state</code>, <code>pending</code>, and <code>message</code>.</li>
<li>Call <code>chitchat_response(message)</code>, storing the result in <code>response</code>. If there is a response, print it and return the <code>state</code> along with <code>None</code>.</li>
<li>Unpack the <code>policy_rules</code> dictionary into the variables <code>new_state</code>, <code>response</code>, and <code>pending_state</code>. To do this, pass in a tuple consisting of <code>state</code> and <code>interpret(message)</code>.</li>
<li>If <code>pending</code> is not none, extract the new states and response by using <code>pending</code> as the key of <code>policy_rules</code>.</li>
</ul>

In [ ]:
# Define send_message()
def send_message(state, pending, message):
    print("USER : {}".format(message))
    response = chitchat_response(message)
    if response is not None:
        print("BOT : {}".format(response))
        return state, None
    
    # Calculate the new_state, response, and pending_state
    new_state, response, pending_state = policy_rules[(state, interpret(message))]
    print("BOT : {}".format(response))
    if pending is not None:
        new_state, response, pending_state = policy_rules[pending]
        print("BOT : {}".format(response))        
    if pending_state is not None:
        pending = (pending_state, interpret(message))
    return new_state, pending

# Define send_messages()
def send_messages(messages):
    state = INIT
    pending = None
    for msg in messages:
        state, pending = send_message(state, pending, msg)

# Send the messages
send_messages([
    "I'd like to order some coffee",
    "555-12345",
    "do you remember when I ordered 1000 kilos by accident?",
    "kenyan"
])  

### Frontiers of dialogue research

### Generating text with neural networks

<div class=""><p>In this final exercise of the course, you're going to generate text using a neural network trained on the scripts of every episode of The Simpsons. Specifically, you'll use a simplified version of the <code>sample_text()</code> function that Alan described in the video.</p>
<p>It takes in two arguments: <code>seed</code> and <code>temperature</code>. The <code>seed</code> argument is the initial sequence that the network uses to generate the subsequent text, while the <code>temperature</code> argument controls how risky the network is when generating text. At very low temperatures, it just repeats the most common combinations  of letters, and at very high temperatures, it generates complete gibberish. In order to ensure fast runtimes, the network in this exercise will only work for a subset of <code>temperature</code> values.</p>
<p>After you finish this exercise, be sure to check out <a href="https://www.datacamp.com/community/tutorials/facebook-chatbot-python-deploy" target="_blank" rel="noopener noreferrer">this tutorial</a> by Alan where he walks you through how to connect a chatbot to Facebook Messenger!</p></div>

In [ ]:
#:S
generated = {0.2: "i'm gonna punch lenny in the back of the been a to the on the man to the mother and the father to simpson the father to with the marge in the for the like the fame to the been to the for my bart the don't was in the like the for the father the father a was the father been a say the been to me the do it and the father been to go. i want to the boy i can the from a man to be the for the been a like the father to make my bart of the father",
 0.5: "i'm gonna punch lenny in the back of the kin't she change and i'm all better it and the was the fad a drivera it? what i want to did hey, he would you would in your bus who know is the like and this don't are for your this all for your manset the for it a man is on the see the will they want to know i'm are for one start of that and i got the better this is. it whoce and i don't are on the mater stop in the from a for the be your mileat",
 1.0: "i'm gonna punch lenny in the back of the to to macks how screath. firl done we wouldn't wil that kill. of this torshmobote since, i know i ord did, can give crika of sintenn prescoam.whover my me after may? there's right. that up. there's ruining isay.oh.solls.nan'h those off point chuncing car your anal medion.hey, are exallies a off while bea dolk of sure, hello, no in her, we'll rundems... i'm eventy taving me to too the letberngonce",
 1.2: "i'm gonna punch lenny in the back of the burear prespe-nakes, 'lisa to isn't that godios.and when be the bowniday' would lochs meine, mind crikvin' suhle ovotaci!..... hey, a poielyfd othe flancer, this in are rightplouten of of we doll hurrs, truelturone? rake inswaydan justy!we scrikent.ow.. by back hous, smadge, the lighel irely.yes, homer. wel'e esasmoy ryelalrs all wronencay...... nank. i wenth makedyk. come on help cerzind, now, n"}
def sample_text(seed, temperature):
    return generated[temperature]

Instructions
<ul>
<li>Set the seed to be <code>"i'm gonna punch lenny in the back of the"</code>.</li>
<li>For each of the riskiness values <code>[0.2, 0.5, 1.0, 1.2]</code>, call the <code>sample_text()</code> function with the arguments <code>seed</code> and <code>temperature</code>.</li>
</ul>

In [ ]:
# Feed the seed text into the neural network
seed = "i'm gonna punch lenny in the back of the"

# Iterate over the different temperature values
for temperature in [0.2, 0.5, 1.0, 1.2]:
    print("\nGenerating text with riskiness : {}\n".format(temperature))
    # Call the sample_text function
    print(sample_text(seed, temperature))

**You just generated some text with a neural network. Scroll through the output to see the text generated with different values of the temperature parameter. And congratulations on completing the course!**